In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:

URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:02012R0432-20210517#tocId5"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [49]:
# Find the <p> tag with class "title-table"
title_table_p = soup.find('p', class_='title-table')

# Find the next <table> tag following the title-table <p> tag
desired_table = title_table_p.find_next('table')

# Replace sub-tables in cells with a placeholder string
for cell in desired_table.find_all('td'):
    sub_table = cell.find('table')
    if sub_table:
        cell.string = 'SUB_TABLE_PLACEHOLDER'

In [50]:
headers = [header.get_text(strip=True) for header in desired_table.find('tr').find_all('td')]

In [60]:
df = pd.read_html(str(desired_table), header=0)[0]
df = df.drop(columns=['Bedingungen und/oder Beschränkungen hinsichtlich der Verwendung des Lebensmittels und/oder zusätzliche Erklärungen oder Warnungen',
                       'Nummer im EFSA Journal',
                       'Nummer des Eintrags in der konsolidierten Liste, die der EFSA zur Bewertung vorgelegt wurde'])
df = df.rename(columns={"Nährstoff, Substanz, Lebensmittel oder Lebensmittelkategorie": "Nährstoff",
                        "Angabe":"Claim",
                   "Bedingungen für die Verwendung der Angabe": "Bedingungen",
                  })

In [67]:
# Define the pattern to look for in the specified columns
pattern_to_drop = "▼"

# Create a boolean mask indicating which rows to drop
rows_to_drop = df.apply(lambda row: any(pattern_to_drop in value for value in row), axis=1)

# Drop the rows from the DataFrame
df_filtered = df[~rows_to_drop]

In [68]:
df_filtered.to_json('claims_list.json', orient='records', lines=True, force_ascii=False)

In [63]:
df

,Nährstoff,Claim,Bedingungen
0,Aktivkohle,Aktivkohle trägt zur Verringerung übermäßiger ...,Die Angabe darf nur für Lebensmittel verwendet...
1,▼M1,▼M1,▼M1
2,Alpha-Cyclodextrin,Der Verzehr von Alpha-Cyclodextrin als Bestand...,Die Angabe darf verwendet werden für Lebensmit...
3,▼B,▼B,▼B
4,Alpha-Linolensäure (ALA),ALA trägt zur Aufrechterhaltung eines normalen...,Die Angabe darf nur für Lebensmittel verwendet...
...,...,...,...
268,Zuckerfreier Kaugummi,Zuckerfreier Kaugummi trägt zur Erhaltung der ...,Die Angabe darf nur für Kaugummi verwendet wer...
269,Zuckerfreier Kaugummi,Zuckerfreier Kaugummi trägt zur Neutralisierun...,Die Angabe darf nur für Kaugummi verwendet wer...
270,Zuckerfreier Kaugummi,Zuckerfreier Kaugummi trägt zur Verringerung v...,Die Angabe darf nur für Kaugummi verwendet wer...
271,Zuckerfreier Kaugummi mit Carbamid,Zuckerfreier Kaugummi mit Carbamid neutralisie...,Die Angabe darf nur für Kaugummi verwendet wer...
